# SVM — Fronteira, *margens* (±1) e *vetores de suporte*

Este notebook treina `SVC` com três kernels (`linear`, `poly`, `rbf`) em três datasets distintos e plota:
- regiões de decisão,
- **fronteira** (`decision_function=0`),
- **margens** (`decision_function=±1`),
- **vetores de suporte** (marcados com `x`).

Os dados são padronizados; os vetores de suporte exibidos estão no **espaço escalonado**.


In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from sklearn.datasets import make_classification, make_circles, make_moons
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.svm import SVC
from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay

def build_linear(n=600, random_state=7):
    X, y = make_classification(n_samples=n, n_features=2, n_informative=2, n_redundant=0,
                               n_clusters_per_class=1, class_sep=2.0, flip_y=0.01, random_state=random_state)
    return X, y

def build_poly(n=600, random_state=7):
    X, y = make_circles(n_samples=n, factor=0.45, noise=0.08, random_state=random_state)
    return X, y

def build_rbf(n=600, random_state=7):
    X, y = make_moons(n_samples=n, noise=0.15, random_state=random_state)
    return X, y

datasets = {
    "linear": build_linear(),
    "polynomial (circles)": build_poly(),
    "rbf (moons)": build_rbf()
}

def split_scale(X, y, test_size=0.3, seed=37):
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=test_size, stratify=y, random_state=seed)
    sc = StandardScaler()
    X_train_s = sc.fit_transform(X_train)
    X_test_s = sc.transform(X_test)
    return X_train_s, X_test_s, y_train, y_test

def plot_decision_with_margins(model, X_train, X_test, y_test, title):
    X_vis = np.vstack([X_train, X_test])
    h = 0.02
    x_min, x_max = X_vis[:, 0].min() - 0.5, X_vis[:, 0].max() + 0.5
    y_min, y_max = X_vis[:, 1].min() - 0.5, X_vis[:, 1].max() + 0.5
    xx, yy = np.meshgrid(np.arange(x_min, x_max, h), np.arange(y_min, y_max, h))
    grid = np.c_[xx.ravel(), yy.ravel()]
    Z = model.decision_function(grid).reshape(xx.shape)
    Z_pred = model.predict(grid).reshape(xx.shape)
    plt.figure(figsize=(6, 5))
    plt.contourf(xx, yy, Z_pred, alpha=0.25)
    CS = plt.contour(xx, yy, Z, levels=[-1, 0, 1], linestyles=['--', '-', '--'])
    plt.clabel(CS, inline=True, fontsize=8)
    plt.scatter(X_test[:, 0], X_test[:, 1], c=y_test, s=22, edgecolor='k')
    sv = model.support_vectors_
    plt.scatter(sv[:, 0], sv[:, 1], s=60, marker='x')
    plt.title(title)
    plt.xlabel("Feature 1 (scaled)")
    plt.ylabel("Feature 2 (scaled)")
    plt.tight_layout()
    plt.show()

C_val, degree_val, gamma_val = 1.0, 2, "scale"
for dname, (X, y) in datasets.items():
    Xtr, Xte, ytr, yte = split_scale(X, y)
    models = {
        "linear": SVC(kernel="linear", C=C_val, decision_function_shape="ovr", random_state=42).fit(Xtr, ytr),
        "poly":   SVC(kernel="poly",   C=C_val, degree=degree_val, gamma=gamma_val,
                      decision_function_shape="ovr", random_state=42).fit(Xtr, ytr),
        "rbf":    SVC(kernel="rbf",    C=C_val, gamma=gamma_val,
                      decision_function_shape="ovr", random_state=42).fit(Xtr, ytr),
    }
    for kernel, clf in models.items():
        y_pred = clf.predict(Xte)
        cm = confusion_matrix(yte, y_pred)
        disp = ConfusionMatrixDisplay(confusion_matrix=cm)
        disp.plot(values_format='d')
        plt.title(f"{dname} — Matriz de Confusão SVC ({kernel})")
        plt.tight_layout()
        plt.show()
        plot_decision_with_margins(clf, Xtr, Xte, yte, f"{dname} — SVC ({kernel}) com margens e SVs")